Import MyDataBase class.

In [1]:
from src.database import MyDataBase
db_name = "parch_and_posey"

Load database.

In [2]:
database = MyDataBase(db_name)
database.print_tables_names()

ISOLATION_LEVEL_AUTOCOMMIT: 0
('web_events',)
('sales_reps',)
('region',)
('orders',)
('accounts',)


### Window Functions

PostgreSQL’s documentation does an excellent job of
[introducing the concept of Window Functions](https://www.postgresql.org/docs/9.1/tutorial-window.html):
a window function performs a calculation across a set of table rows
that are somehow related to the current row.
This is comparable to the type of calculation that can be done with an aggregate function.
But unlike regular aggregate functions,
use of a window function does not cause rows to become grouped into a single output row —
the rows retain their separate identities. Behind the scenes,
the window function is able to access more than just the current row of the query result.

Through introducing window functions,
we have also introduced two statements that you may not be familiar with:
OVER and PARTITION BY.
These are key to window functions.
Not every window function uses PARTITION BY;
we can also use ORDER BY or no statement at all depending on the query we want to run.
You will practice using these clauses in the upcoming quizzes.
If you want more details right now,
[this resource](https://blog.sqlauthority.com/2015/11/04/sql-server-what-is-the-over-clause-notes-from-the-field-101/)
from Pinal Dave is helpful.

Note: You can’t use window functions and standard aggregations in the same query.
More specifically, you can’t include window functions in a GROUP BY clause.

(Source: Udacity SQL Course)

Running total of how much standard paper was sold per day.

In [3]:
q = """
SELECT standard_qty,
    DATE_TRUNC('month', occurred_at) AS month,
    SUM(standard_qty) OVER (ORDER BY occurred_at) AS running_total
FROM orders;
"""

table = database.to_data_frame(q)
table

,standard_qty,month,running_total
0,0,2013-12-01,0
1,490,2013-12-01,490
2,528,2013-12-01,1018
3,0,2013-12-01,1018
4,492,2013-12-01,1510
...,...,...,...
6907,0,2017-01-01,1937478
6908,497,2017-01-01,1937975
6909,38,2017-01-01,1938013
6910,291,2017-01-01,1938304


Running total of how much standard paper was sold starting at the beginning of each month.

In [4]:
q = """
SELECT standard_qty,
    DATE_TRUNC('month', occurred_at) AS month,
    SUM(standard_qty) OVER (PARTITION BY DATE_TRUNC('month', occurred_at) ORDER BY occurred_at) AS running_total
FROM orders;
"""

table = database.to_data_frame(q)
table

,standard_qty,month,running_total
0,0,2013-12-01,0
1,490,2013-12-01,490
2,528,2013-12-01,1018
3,0,2013-12-01,1018
4,492,2013-12-01,1510
...,...,...,...
6907,0,2017-01-01,4417
6908,497,2017-01-01,4914
6909,38,2017-01-01,4952
6910,291,2017-01-01,5243


Without ***ORDER BY*** each value will simply be a sum of all the *standard_qty* values
in the respective month.

In [5]:
q = """
SELECT standard_qty,
    DATE_TRUNC('month', occurred_at) AS month,
    SUM(standard_qty) OVER (PARTITION BY DATE_TRUNC('month', occurred_at)) AS running_total
FROM orders;
"""

table = database.to_data_frame(q)
table


,standard_qty,month,running_total
0,497,2013-12-01,26554
1,508,2013-12-01,26554
2,84,2013-12-01,26554
3,0,2013-12-01,26554
4,304,2013-12-01,26554
...,...,...,...
6907,485,2017-01-01,5285
6908,42,2017-01-01,5285
6909,495,2017-01-01,5285
6910,52,2017-01-01,5285


### Creating a Running Total Using Window Functions

Create a running total of *standard_amt_usd* (in the *orders* table)
over order time with no date truncation.
Your final table should have two columns:
one with the amount being added for each new row,
and a second with the running total.

In [6]:
q = """
SELECT standard_amt_usd,
    SUM(standard_amt_usd) OVER (ORDER BY occurred_at) AS running_total
FROM orders;
"""

table = database.to_data_frame(q)
table



,standard_amt_usd,running_total
0,0.00,0.00
1,2445.10,2445.10
2,2634.72,5079.82
3,0.00,5079.82
4,2455.08,7534.90
...,...,...
6907,0.00,9668015.22
6908,2480.03,9670495.25
6909,189.62,9670684.87
6910,1452.09,9672136.96


### Creating a *Partitioned* Running Total Using Window Functions

Now, modify your query from the previous quiz to include partitions.
Still create a running total of *standard_amt_usd* (in the *orders* table) over order time,
but this time, date truncate *occurred_at* by year
and partition by that same year-truncated *occurred_at* variable.
Your final table should have three columns: One with the amount being added for each row,
one for the truncated date, and a final column with the running total within each year.

In [7]:
q = """
SELECT standard_amt_usd,
       DATE_TRUNC('year', occurred_at) as year,
       SUM(standard_amt_usd) OVER (PARTITION BY DATE_TRUNC('year', occurred_at) ORDER BY occurred_at) AS running_total
FROM orders
"""

table = database.to_data_frame(q)
table

,standard_amt_usd,year,running_total
0,0.00,2013-01-01,0.00
1,2445.10,2013-01-01,2445.10
2,2634.72,2013-01-01,5079.82
3,0.00,2013-01-01,5079.82
4,2455.08,2013-01-01,7534.90
...,...,...,...
6907,0.00,2017-01-01,22040.83
6908,2480.03,2017-01-01,24520.86
6909,189.62,2017-01-01,24710.48
6910,1452.09,2017-01-01,26162.57


Add row number.

In [8]:
q = """
SELECT id,
      account_id,
      occurred_at,
      ROW_NUMBER() OVER (ORDER BY id) AS row_num
FROM orders;
"""

table = database.to_data_frame(q)
table

,id,account_id,occurred_at,row_num
0,1,1001,2015-10-06 17:31:14,1
1,2,1001,2015-11-05 03:34:33,2
2,3,1001,2015-12-04 04:21:55,3
3,4,1001,2016-01-02 01:18:24,4
4,5,1001,2016-02-01 19:27:27,5
...,...,...,...,...
6907,6908,4501,2016-06-29 04:03:39,6908
6908,6909,4501,2016-07-29 19:58:32,6909
6909,6910,4501,2016-08-27 00:58:11,6910
6910,6911,4501,2016-11-22 06:52:22,6911


Start count over at each partition.

In [9]:
q = """
SELECT id,
      account_id,
      occurred_at,
      ROW_NUMBER() OVER (PARTITION BY account_id ORDER BY id) AS row_num
FROM orders;
"""

table = database.to_data_frame(q)
table

,id,account_id,occurred_at,row_num
0,1,1001,2015-10-06 17:31:14,1
1,2,1001,2015-11-05 03:34:33,2
2,3,1001,2015-12-04 04:21:55,3
3,4,1001,2016-01-02 01:18:24,4
4,5,1001,2016-02-01 19:27:27,5
...,...,...,...,...
6907,6908,4501,2016-06-29 04:03:39,9
6908,6909,4501,2016-07-29 19:58:32,10
6909,6910,4501,2016-08-27 00:58:11,11
6910,6911,4501,2016-11-22 06:52:22,12


***RANK*** is used to assign a rank to every row within a partition of a result set.
The ***RANK()*** function adds the number of tied rows to the tied rank
to calculate the rank of the next row,
so the ranks may not be sequential.
In addition, rows with the same values will get the same rank.

In [10]:
q = """
SELECT id,
      account_id,
      DATE_TRUNC('month', occurred_at) AS month,
      RANK() OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month', occurred_at)) AS row_num
FROM orders;
"""

table = database.to_data_frame(q)
table

,id,account_id,month,row_num
0,1,1001,2015-10-01,1
1,4307,1001,2015-11-01,2
2,2,1001,2015-11-01,2
3,3,1001,2015-12-01,4
4,4308,1001,2015-12-01,4
...,...,...,...,...
6907,4304,4501,2016-10-01,9
6908,4305,4501,2016-11-01,10
6909,6911,4501,2016-11-01,10
6910,6912,4501,2016-12-01,12


***DENSE_RANK()*** does not skip values after assigning several rows with the same rank.

In [11]:
q = """
SELECT id,
      account_id,
      DATE_TRUNC('month', occurred_at) AS month,
      DENSE_RANK() OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month', occurred_at)) AS row_num
FROM orders;
"""

table = database.to_data_frame(q)
table


,id,account_id,month,row_num
0,1,1001,2015-10-01,1
1,4307,1001,2015-11-01,2
2,2,1001,2015-11-01,2
3,3,1001,2015-12-01,3
4,4308,1001,2015-12-01,3
...,...,...,...,...
6907,4304,4501,2016-10-01,6
6908,4305,4501,2016-11-01,7
6909,6911,4501,2016-11-01,7
6910,6912,4501,2016-12-01,8


### Ranking Total Paper Ordered by Account

Select the `id`, `account_id`, and `total` variable from the `orders` table,
then create a column called `total_rank` that ranks this total amount of paper
ordered (from highest to lowest) for each account using a partition.
Your final table should have these four columns.

In [12]:
q = """
SELECT id,
       account_id,
       total,
       RANK() OVER (PARTITION BY account_id ORDER BY total DESC) AS total_rank
FROM orders
"""

table = database.to_data_frame(q)
table

,id,account_id,total,total_rank
0,4308,1001,1410,1
1,4309,1001,1405,2
2,4316,1001,1384,3
3,4317,1001,1347,4
4,4314,1001,1343,5
...,...,...,...,...
6907,4301,4501,164,9
6908,4300,4501,149,10
6909,4306,4501,126,11
6910,4299,4501,26,12


Aggregates in Window Functions with and without ***ORDER BY***.

In [13]:
q = """
SELECT id,
       account_id,
       standard_qty,
       DATE_TRUNC('month', occurred_at) AS month,
       DENSE_RANK() OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS dense_rank,
       SUM(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS sum_std_qty,
       COUNT(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS count_std_qty,
       AVG(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS avg_std_qty,
       MIN(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS min_std_qty,
       MAX(standard_qty) OVER (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at)) AS max_std_qty
FROM orders
"""

table = database.to_data_frame(q)
table


,id,account_id,standard_qty,month,dense_rank,sum_std_qty,count_std_qty,avg_std_qty,min_std_qty,max_std_qty
0,1,1001,123,2015-10-01,1,123,1,123.000000,123,123
1,4307,1001,506,2015-11-01,2,819,3,273.000000,123,506
2,2,1001,190,2015-11-01,2,819,3,273.000000,123,506
3,3,1001,85,2015-12-01,3,1430,5,286.000000,85,526
4,4308,1001,526,2015-12-01,3,1430,5,286.000000,85,526
...,...,...,...,...,...,...,...,...,...,...
6907,4304,4501,159,2016-10-01,6,759,9,84.333333,5,180
6908,4305,4501,6,2016-11-01,7,828,11,75.272727,5,180
6909,6911,4501,63,2016-11-01,7,828,11,75.272727,5,180
6910,6912,4501,61,2016-12-01,8,1015,13,78.076923,5,180


In [14]:
q = """
SELECT id,
       account_id,
       standard_qty,
       occurred_at,
       DENSE_RANK() OVER (PARTITION BY account_id) AS dense_rank,
       SUM(standard_qty) OVER (PARTITION BY account_id) AS sum_std_qty,
       COUNT(standard_qty) OVER (PARTITION BY account_id) AS count_std_qty,
       AVG(standard_qty) OVER (PARTITION BY account_id) AS avg_std_qty,
       MIN(standard_qty) OVER (PARTITION BY account_id) AS min_std_qty,
       MAX(standard_qty) OVER (PARTITION BY account_id) AS max_std_qty
FROM orders
"""

table = database.to_data_frame(q)
table

,id,account_id,standard_qty,occurred_at,dense_rank,sum_std_qty,count_std_qty,avg_std_qty,min_std_qty,max_std_qty
0,14,1001,97,2016-10-26 20:31:30,1,7896,28,282.000000,85,566
1,4318,1001,485,2016-11-25 23:19:37,1,7896,28,282.000000,85,566
2,4317,1001,507,2016-09-26 23:22:47,1,7896,28,282.000000,85,566
3,4316,1001,557,2016-08-28 06:50:58,1,7896,28,282.000000,85,566
4,4315,1001,457,2016-07-30 03:21:57,1,7896,28,282.000000,85,566
...,...,...,...,...,...,...,...,...,...,...
6907,4299,4501,15,2016-05-30 04:18:34,1,1015,13,78.076923,5,180
6908,4303,4501,158,2016-09-25 01:44:03,1,1015,13,78.076923,5,180
6909,4304,4501,159,2016-10-24 08:50:37,1,1015,13,78.076923,5,180
6910,4305,4501,6,2016-11-22 06:57:04,1,1015,13,78.076923,5,180


### Aggregates in Window Functions with and without ORDER BY

The ***ORDER BY*** clause is one of two clauses integral to window functions.
The ***ORDER*** and ***PARTITION*** define what is referred to as the “window” —
the ordered subset of data over which calculations are made.
Removing ***ORDER BY*** just leaves an unordered partition;
in our query's case, each column's value is simply an aggregation
(e.g., sum, count, average, minimum, or maximum)
of all the `standard_qty` values in its respective `account_id`.

As Stack Overflow user
[mathguy](https://stackoverflow.com/questions/41364665/analytic-count-over-partition-with-and-without-order-by-clause)
explains:

The easiest way to think about this - leaving the ***ORDER BY*** out is equivalent to
"ordering" in a way that all rows in the partition are "equal" to each other.
Indeed, you can get the same effect by explicitly adding the ***ORDER BY*** clause like this:
***ORDER BY 0*** (or "order by" any constant expression), or even, more emphatically,
***ORDER BY NULL***.

You can check out a complete list of window functions in Postgres (the syntax Mode uses) in the
[Postgres documentation](https://www.postgresql.org/docs/8.4/functions-window.html).

In [15]:
q = """
SELECT id,
       account_id,
       standard_qty,
       DATE_TRUNC('month', occurred_at) AS month,
       DENSE_RANK() OVER main_window AS dense_rank,
       SUM(standard_qty) OVER main_window AS sum_std_qty,
       COUNT(standard_qty) OVER main_window AS count_std_qty,
       AVG(standard_qty) OVER main_window AS avg_std_qty,
       MIN(standard_qty) OVER main_window AS min_std_qty,
       MAX(standard_qty) OVER main_window AS max_std_qty
FROM orders
WINDOW main_window AS (PARTITION BY account_id ORDER BY DATE_TRUNC('month',occurred_at))
"""

table = database.to_data_frame(q)
table

,id,account_id,standard_qty,month,dense_rank,sum_std_qty,count_std_qty,avg_std_qty,min_std_qty,max_std_qty
0,1,1001,123,2015-10-01,1,123,1,123.000000,123,123
1,4307,1001,506,2015-11-01,2,819,3,273.000000,123,506
2,2,1001,190,2015-11-01,2,819,3,273.000000,123,506
3,3,1001,85,2015-12-01,3,1430,5,286.000000,85,526
4,4308,1001,526,2015-12-01,3,1430,5,286.000000,85,526
...,...,...,...,...,...,...,...,...,...,...
6907,4304,4501,159,2016-10-01,6,759,9,84.333333,5,180
6908,4305,4501,6,2016-11-01,7,828,11,75.272727,5,180
6909,6911,4501,63,2016-11-01,7,828,11,75.272727,5,180
6910,6912,4501,61,2016-12-01,8,1015,13,78.076923,5,180


Another example:

In [16]:
q = """
SELECT id,
       account_id,
       DATE_TRUNC('year',occurred_at) AS year,
       DENSE_RANK() OVER account_year_window AS dense_rank,
       total_amt_usd,
       SUM(total_amt_usd) OVER account_year_window AS sum_total_amt_usd,
       COUNT(total_amt_usd) OVER account_year_window AS count_total_amt_usd,
       AVG(total_amt_usd) OVER account_year_window AS avg_total_amt_usd,
       MIN(total_amt_usd) OVER account_year_window AS min_total_amt_usd,
       MAX(total_amt_usd) OVER account_year_window AS max_total_amt_usd
FROM orders
WINDOW account_year_window AS (PARTITION BY account_id ORDER BY DATE_TRUNC('year',occurred_at))
"""

table = database.to_data_frame(q)
table

,id,account_id,year,dense_rank,total_amt_usd,sum_total_amt_usd,count_total_amt_usd,avg_total_amt_usd,min_total_amt_usd,max_total_amt_usd
0,4307,1001,2015-01-01,1,8757.18,21651.53,5,4330.306,776.18,9426.71
1,3,1001,2015-01-01,1,776.18,21651.53,5,4330.306,776.18,9426.71
2,2,1001,2015-01-01,1,1718.03,21651.53,5,4330.306,776.18,9426.71
3,1,1001,2015-01-01,1,973.43,21651.53,5,4330.306,776.18,9426.71
4,4308,1001,2015-01-01,1,9426.71,21651.53,5,4330.306,776.18,9426.71
...,...,...,...,...,...,...,...,...,...,...
6907,6910,4501,2016-01-01,1,1449.74,14629.16,13,1125.320,86.78,2024.48
6908,6911,4501,2016-01-01,1,1473.92,14629.16,13,1125.320,86.78,2024.48
6909,4306,4501,2016-01-01,1,628.74,14629.16,13,1125.320,86.78,2024.48
6910,4305,4501,2016-01-01,1,86.78,14629.16,13,1125.320,86.78,2024.48


### Scenarios for using ***LAG*** and ***LEAD*** functions

You can use ***LAG*** and ***LEAD*** functions whenever you are trying to compare the values in adjacent rows
or rows that are offset by a certain number.

Comparing a Row to Previous Row

Imagine you're an analyst at Parch & Posey and you want to determine how the current order's total revenue
("total" meaning from sales of all types of paper) compares to the next order's total revenue.

In [17]:
q = """
SELECT day,
       total_amt_usd,
       LEAD(total_amt_usd) OVER (ORDER BY day) AS lead,
       LEAD(total_amt_usd) OVER (ORDER BY day) - total_amt_usd AS lead_difference
FROM (
SELECT DATE_TRUNC('day', occurred_at) AS day,
       SUM(total_amt_usd) AS total_amt_usd
  FROM orders
 GROUP BY 1
 ORDER BY DATE_TRUNC('day', occurred_at)
) sub
"""

table = database.to_data_frame(q)
table

,day,total_amt_usd,lead,lead_difference
0,2013-12-04,5983.87,3278.98,-2704.89
1,2013-12-05,3278.98,24071.14,20792.16
2,2013-12-06,24071.14,58294.79,34223.65
3,2013-12-08,58294.79,5219.81,-53074.98
4,2013-12-09,5219.81,7266.54,2046.73
...,...,...,...,...
1055,2016-12-29,43364.81,21914.50,-21450.31
1056,2016-12-30,21914.50,63505.30,41590.80
1057,2016-12-31,63505.30,71699.67,8194.37
1058,2017-01-01,71699.67,6451.76,-65247.91


### Percentiles with Partitions

1. Use the ***NTILE*** functionality to divide the accounts into 4 levels in terms of the amount of `standard_qty`
for their orders. Your resulting table should have the `account_id`, the `occurred_at` time for each order,
the total amount of `standard_qty` paper purchased, and one of four levels in a `standard_quartile` column.

In [18]:
q = """
SELECT
       account_id,
       occurred_at,
       standard_qty,
       NTILE(4) OVER (PARTITION BY account_id ORDER BY standard_qty) AS standard_quartile
  FROM orders
 ORDER BY account_id DESC
"""

table = database.to_data_frame(q)
table

,account_id,occurred_at,standard_qty,standard_quartile
0,4501,2016-08-27 00:58:11,16,2
1,4501,2016-12-21 13:30:42,61,2
2,4501,2016-11-22 06:52:22,63,2
3,4501,2016-06-29 03:57:11,104,3
4,4501,2016-07-29 20:06:39,111,3
...,...,...,...,...
6907,1001,2016-02-01 19:07:32,473,3
6908,1001,2016-07-30 03:21:57,457,3
6909,1001,2015-11-05 03:34:33,190,3
6910,1001,2016-01-02 01:18:24,144,3


2. Use the ***NTILE*** functionality to divide the accounts into two levels in terms of
the amount of `gloss_qty` for their orders.
Your resulting table should have the `account_id`, the `occurred_at` time for each order,
the total amount of `gloss_qty` paper purchased, and one of two levels in a `gloss_half` column.


In [19]:
q = """
SELECT
       account_id,
       occurred_at,
       gloss_qty,
       NTILE(2) OVER (PARTITION BY account_id ORDER BY gloss_qty) AS gloss_half
  FROM orders
 ORDER BY account_id DESC
"""

table = database.to_data_frame(q)
table

,account_id,occurred_at,gloss_qty,gloss_half
0,4501,2016-08-27 00:48:17,11,1
1,4501,2016-05-30 04:18:34,11,1
2,4501,2016-06-29 03:57:11,14,1
3,4501,2016-07-29 20:06:39,16,2
4,4501,2016-11-22 06:52:22,67,2
...,...,...,...,...
6907,1001,2016-11-25 23:19:37,543,2
6908,1001,2016-07-30 03:21:57,532,2
6909,1001,2016-10-26 20:31:30,143,2
6910,1001,2016-12-24 05:53:13,127,2


3. Use the ***NTILE*** functionality to divide the orders for each account into 100 levels in terms of
the amount of `total_amt_usd` for their orders.
Your resulting table should have the `account_id`, the `occurred_at` time for each order,
the total amount of `total_amt_usd` paper purchased, and one of 100 levels in a `total_percentile` column.

In [20]:
q = """
SELECT
       account_id,
       occurred_at,
       total_amt_usd,
       NTILE(100) OVER (PARTITION BY account_id ORDER BY total_amt_usd) AS total_percentile
  FROM orders
 ORDER BY account_id DESC
"""

table = database.to_data_frame(q)
table

,account_id,occurred_at,total_amt_usd,total_percentile
0,4501,2016-07-29 20:06:39,974.17,5
1,4501,2016-10-24 08:50:37,1122.55,6
2,4501,2016-08-27 00:48:17,1175.47,7
3,4501,2016-09-25 01:44:03,1324.34,8
4,4501,2016-08-27 00:58:11,1449.74,9
...,...,...,...,...
6907,1001,2016-07-30 03:21:57,8286.99,18
6908,1001,2016-11-25 23:19:37,7924.46,17
6909,1001,2016-05-01 15:55:51,2052.20,16
6910,1001,2016-10-26 20:31:30,1993.58,15


In [21]:
database.close()